In [ ]:
%load_ext autoreload
%autoreload 2

from condorgmm.data.mp4_vda import MP4DepthAnythingVideo

In [ ]:
og_video = MP4DepthAnythingVideo(
    "/home/georgematheos/condorgmm/assets/custom/room_spin.MOV",
    min_depth_meters=1.25,
    max_depth_meters=3.0,
    encoder="vitl",
)
video = og_video.downscale(6)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(video[0].rgb)

In [ ]:
plt.imshow(video[0].depth)
plt.colorbar()

In [ ]:
import genjax
genjax.pretty()

In [ ]:
from condorgmm.condor.interface.camera_tracking import initialize, update, fast_config
import rerun as rr
from condorgmm.condor.rerun import log_state
from tqdm import tqdm
import jax.numpy as jnp
import condorgmm


def _f(x):
    return jnp.array(x, dtype=jnp.float32)

hyp = fast_config.base_hypers

cfg = fast_config.replace(
    base_hypers=hyp,
    n_gaussians=300,
    tile_size_x=8,
    tile_size_y=8,
    step_n_sweeps_phase_1=4,
    repopulate_depth_nonreturns=False
)
scenedepth = jnp.sum(video[0].depth) / jnp.sum(video[0].depth > 0)
scenepose = condorgmm.Pose(jnp.array([0., 0., scenedepth, 1., 0., 0., 0.], dtype=jnp.float32))

In [ ]:
condorgmm.rr_init("mp4-test-00")

In [ ]:

for i in tqdm(range(10)):
    if i == 0:
        _, ccts = initialize(video[i], scenepose, cfg)
    else:
        _, ccts = update(video[i], scenepose, ccts, cfg, get_gmm=False)

    rr.set_time_sequence("frame", i)
    log_state(ccts.state, ccts.hypers)
    rr.log("depth_img/observation", rr.DepthImage(video[i].depth))
    rr.log("depth_img/inferred", rr.DepthImage(ccts.state.datapoints.value.xyz[..., 2].reshape(video[i].depth.shape)))